In [ ]:
import numpy as np
import matplotlib.pyplot as plt

1.


   To find the minimizer, we need to take the gradient of $( f(x) $) and set it equal to zero:
   
   $\nabla f(x) = \begin{bmatrix} 2x_1 + 4x_2 \\ 4x_1 + 8x_2 \end{bmatrix} = \begin{bmatrix} 0 \\ 0 \end{bmatrix}
   $ \
   Solving this system of equations, we get $( x_1 = 0 ) and ( x_2 = 0 )$. So, the minimizer of $( f(x) $) is $( x^* = \begin{bmatrix} 0 \\ 0 \end{bmatrix} $).

   The minimum function value is obtained by substituting $( x^* $) into $( f(x) $): \
   $
   f(x^*) = (0)^2 + 4(0)(0) + 4(0)^2 = 0
   $


   In this case, the minimizer $( x^* = \begin{bmatrix} 0 \\ 0 \end{bmatrix} ) $ \
    is unique, as there is only one solution to the system of equations.


   The point $( x^* = \begin{bmatrix} 0 \\ 0 \end{bmatrix} ) $ is a global minimum since the function $( f(x) )$ is a quadratic function, and the minimum value occurs at the origin.

 The function $( f(x) $) is convex because its Hessian matrix (second-order partial derivatives) is positive definite. \
    The Hessian matrix is:
     \
      H = \begin{bmatrix} 2 & 4 \\ 4 & 8 \end{bmatrix}
      
     The eigenvalues of this matrix are (10,0)

2.

In [ ]:
def f(x):
  f= x[0]**2+4*x[0]*x[1]+4*x[1]**2
  return f
def grad_fx(x):
  grad=[2*x[0]+4*x[1],8*x[1]+4*x[0]]
  return np.array(grad)
def hes_fx(x):
  hes=[[2,4],[4,8]]
  return np.array(hes)


In [ ]:
x = [3, 4, 4, 4]
lis = [0] * len(x)
lis[1] = x[-1]
print(lis)
dk = np.outer(np.array(lis), np.array(lis))
dk


The function $g(x) = \mathbb{1}_{L(x)} $ is defined based on the set \( L(x) \) as follows:

$
\mathbb{1}_{L(x)} = \begin{cases}
0 & \text{if } x \text{ belongs to } L \\
\infty & \text{if } x \text{ does not belong to } L
\end{cases}
$

The minimizer of this function depends on the nature of the set \( L(x) \). If \( L(x) \) is empty, there exists no point where \( g(x) \) equals 0, and therefore no minimizer exists. However, if \( L(x) \) is not empty, any point within \( L(x) \) minimizes the function since \( g(x) \) attains the value of 0 at such points.

If \( L(x) \) is non-empty, the function's value at its minimizer is 0.

It's worth noting that the function yields an infinite value outside of \( L(x) \), making the concept of minimizer meaningful only when \( L(x) \) is non-empty.


In [ ]:
def coordinate_descent(x):
  k=0
  iteration=[]
  tau=1e-10
  norm=np.linalg.norm(grad_fx(x))

  while norm>tau:
    eta=0.5
    index=np.random.randint(len(x))
    x_i=x[index]
    print(index)
    lis=[0 for i in range(len(x))]
    lis[index]=1
    xnew=x[index]-eta*(grad_fx(x)@np.transpose(lis))
    print(grad_fx(x)@np.transpose(lis))
    x[index]=xnew
    norm=np.linalg.norm(grad_fx(x))
    print(x)
    print(f(x))
    k=k+1
  iteration.append(k)
  return iteration
coordinate_descent([2,2])

In [ ]:
def Dk(x):
  dk=np.outer(np.array(x),np.transpose(x))
  return dk
Dk([0.0,1.0])

**3**

In [ ]:
def get_alpha_with_scaling(xk,index,x_i, alpha0, rho, gamma, Dk):
  alpha = alpha0
  pk = -grad_fx(xk)
  while f(xk + alpha*Dk@pk) > (f(xk) + gamma*alpha*grad_fx(xk)@Dk@pk):
    alpha = rho*alpha
  return alpha
#with scale back track
def coordinate_descent_with_scale(x0):
  x_all=[]
  iteration=[]
  optimizers=[]
  optimal_value=[]

  rho=[0.9, 0.8, 0.75, 0.6, 0.5, 0.4, 0.25, 0.1, 0.01]
  for r in rho:
    tau=1e-5
    x=np.copy(np.array(x0))
    pk=grad_fx(x)
    k=0
    gamma=0.5
    dk=np.array(Dk(x))
    cor=[]
    cor.append(x)

    norm=np.linalg.norm(pk)
    while norm>tau:
      index=np.random.randint(len(x))
      lis=[0 for i in range(len(x))]
      lis[index]=1
      dk=Dk(lis)

      x_i=x[index]

      alpha=get_alpha_with_scaling(x,index,x_i, 1, r, 0.5, dk)
      grad=grad_fx(x)

      x=(x-alpha*(dk@grad))#x[index]=((x_i)-(alpha*grad[index]))
      cor.append(x)
      pk=grad_fx(x)
      norm=np.linalg.norm(pk)
      k=k+1
    optimizers.append(x)
    iteration.append(k)
    x_all.append(cor)
    optimal_value.append(f(x))
  return iteration,optimizers,optimal_value,x_all

iteration,optimizers,optimal_value,x_all=coordinate_descent_with_scale([1.,40.])

In [ ]:
rho=[0.9, 0.8, 0.75, 0.6, 0.5, 0.4, 0.25, 0.1, 0.01]
for i in range(len(rho)):
  print(f'Rho:{rho[i]} no of iteration: {iteration[i]} optimizers: {optimizers[i]} optimal_value: {optimal_value[i]}')
  print('--------------------------------------------------------------------------')


In [ ]:
plt.plot(rho, iteration, marker = 'o', markersize = 5)

The observed behavior in the plot suggests that as the value of rho increases, the number of iterations required for convergence decreases. This trend is expected because a higher value of rho in the backtracking line search algorithm leads to a larger step size being chosen more frequently, allowing the algorithm to converge faster.
However, the slight zig-zag pattern in the plot may be attributed to variations in the landscape of the objective function and the stochastic nature of the coordinate descent algorithm, where different random coordinate selections can lead to fluctuations in convergence behavior.The decreasing trend in the number of iterations with increasing rho indicates that a higher value of rho is beneficial for achieving convergence more efficiently in the coordinate descent algorithm with line search.

In [ ]:
from matplotlib import cm
x = np.linspace(-100, 80, 1000)
y = np.linspace(-100, 80, 1000)
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(X)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i, j] = f([X[i, j], Y[i, j]])

for i in range(len(x_all)):
    cor = x_all[i]
    xcordinates1 = [xk[0] for xk in cor]
    ycordinates1 = [xk[1] for xk in cor]
    plt.plot(xcordinates1, ycordinates1, label='rho=' + str(rho[i]))

plt.contour(X, Y, Z, cmap=cm.PuBu_r)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Level Sets and Trajectory')
plt.legend()
plt.show()


The proximal operator of a function is a key concept in convex optimization and plays a significant role in solving optimization problems involving nonsmooth functions. The proximal operator of a function $ h(y) $, denoted as $ \text{prox}_\lambda h(x) $, computes the point that minimizes the sum of the function $ h(y) $ and the squared distance to a given point $ x $, weighted by a parameter $ \lambda $. Mathematically, it is defined as:

$\text{prox}_\lambda h(x) = \arg \min_y \left( h(y) + \frac{1}{2\lambda} \| y - x \|^2 \right) $

Now, considering the linearization of $ h(y) $ around a fixed point $ x $, denoted as $ t(y) $, the proximal operator $ \text{prox}_\lambda t(x) $ evaluates to:

$ \text{prox}_\lambda t(x) = \arg \min_y \left( t(y) + \frac{1}{2\lambda} \| y - x \|^2 \right) $

Since $ t(y) $ is the linearization of $ h(y) $ around $ x $, it means that $ t(y) $ approximates $ h(y) $ in the neighborhood of $ x $. Therefore, solving the proximal operator for $ t(x) $ effectively minimizes a quadratic approximation of $ h(y) $ around $ x $, subject to the additional term $ \frac{1}{2\lambda} \| y - x \|^2 $.

Whether this result looks familiar depends on the specific function $ h(y) $ and its linearization $ t(y) $, as well as the context in which it is being applied. In general, the proximal operator plays a crucial role in various optimization algorithms, such as proximal gradient methods and alternating direction method of multipliers (ADMM). The result obtained from evaluating the proximal operator for a linearized function can provide insights into the behavior of these algorithms and how they handle nonsmooth functions in optimization problems.


In [ ]:
def f1(x):
  f=(x[0]-1)**2+x[1]**2+(x[2]+1)**2
  return f
def grad_fx1(x):
  grad=[2*(x[0]-1),2*x[1],2*(x[2]+1)]
  return np.array(grad)
def prox(x):
  y=[]
  for i in x:
    y.append(max(i,0))
  return y

In [ ]:
def proximal_gradient_descent(x,T,eta):
  t=0
  iteration=[]
  optimal=[]
  optimizers=[]
  x_all=[]
  while t<T:
    grad=grad_fx1(x)
    y=x-eta*grad
    x=prox(y)
    t=t+1
    x_all.append(x)
  optimal.append(f1(x))
  optimizers.append(x)
  iteration.append(t)
  return iteration,optimizers,optimal,x_all

In [ ]:
max_itr=[10**2, 500, 10**3, 5000, 10**4, 50000, 10**5, 500000, 10**6,5000000]
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for T in max_itr:
  x=[0.,0.,0.]
  eta=0.3
  iteration,optimizers,optimal,x_all=proximal_gradient_descent(x,T,eta)
  print(f'T:{T}  optimizers: {optimizers} optimal_value: {optimal}')
  print('--------------------------------------------------------------------------')
  xcordinates1 = [xk[0] for xk in x_all]
  ycordinates1 = [xk[1] for xk in x_all]
  zcordinates1 = [xk[2] for xk in x_all]
  ax.plot(xcordinates1, ycordinates1, zcordinates1)

# Plotting the level sets
x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(X)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i, j] = f1([X[i, j], Y[i, j], 0])

ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.5)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()




To comment on the minimizers and objective function values obtained for different choices of the $T$ values, we need to analyze the results obtained from running the proximal gradient descent algorithm with varying $T$ values.

Observation:
Minimizers: As $T$ increases, the algorithm has more iterations to converge towards a solution. Therefore, we would expect that with higher $T$ values, the algorithm will likely reach closer to the true minimizer of the objective function $f(x)$.
Objective Function Values: The objective function value (optimal) obtained after each run of the proximal gradient descent algorithm with different $T$ values provides insight into the quality of the solution achieved. Lower objective function values indicate better solutions. We would expect to see lower optimal values as $T$ increases since the algorithm has more iterations to minimize the objective function.
Trajectory: Plotting the trajectory of the optimization (the path taken by the algorithm to reach the solution) on the level sets of the function $f(x)$ helps visualize how the algorithm navigates the search space. It can show whether the algorithm converges quickly or gets stuck in local minima.


Regarding the question about gradient descent with $\eta = 0.7$,  However, changing the value of $\eta$ may affect the convergence speed and the final solution obtained by the gradient descent algorithm. In general, a larger $\eta$ may lead to faster convergence but may also result in overshooting or divergence. Therefore, it's essential to choose an appropriate value of $\eta$ based on the specific problem and characteristics of the objective function.

To visualize the level sets of the function $f(x)$ and the trajectory of optimization for different $T$ values,we can modify the provided code to include plotting routines for each $T$ value and then observe how the trajectory changes with varying $T$. Additionally, can experiment with different values of $\eta$, such as $0.7$, to observe its effect on the optimization process.




**5**

In [ ]:
def f2(x):
  f=100*(x[1]-x[0]**2)**2+(0.5-x[0])**2
  return f
def grad_fx2(x):
  grad=[-400*(x[1]-x[0]**2)*x[0]-2*(0.5-x[0]),200*(x[1]-x[0]**2)]
  return np.array(grad)
def proxy2(x):

  if np.linalg.norm(x)<=1:
    return x
  else :
    x=(x/np.linalg.norm(x))
    return x

In [ ]:
def proximal_gradient_descent_fx2(x,T,eta):
  t=0
  iteration=[]
  optimal=[]
  optimizers=[]
  x_all=[]
  while t<T:
    grad=grad_fx2(x)
    y=x-eta*grad
    x=proxy2(y)
    t=t+1
    x_all.append(x)
  optimal.append(f2(x))
  optimizers.append(x)
  iteration.append(t)
  return iteration,optimizers,optimal,x_all



In [ ]:
max_itr=[10**2, 500, 10**3, 5000, 10**4, 50000, 10**5, 500000, 10**6,5000000]
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for T in max_itr:
  x=[0.,0.0]
  eta=0.3
  iteration,optimizers,optimal,x_all=proximal_gradient_descent_fx2(x,T,eta)
  print(f'T:{T}  optimizers: {optimizers} optimal_value: {optimal}')
  print('--------------------------------------------------------------------------')
  xcordinates1 = [xk[0] for xk in x_all]
  ycordinates1 = [xk[1] for xk in x_all]

  ax.plot(xcordinates1, ycordinates1)

# Plotting the level sets
x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(X)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i, j] = f2([X[i, j], Y[i, j], 0])

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Level Sets and Trajectory for f2')
plt.colorbar(p)
plt.legend()
plt.show()

To analyze the observations and comment on the minimizers and objective function values obtained for different choices of the T values, let's examine the results obtained from running the proximal gradient descent algorithm with varying T values.

Observations:

Minimizers: As T increases, indicating more iterations of the proximal gradient descent algorithm, the algorithm has more opportunities to converge towards the optimal solution. Therefore, we would expect that with higher T values, the algorithm will likely converge closer to the true minimizer of the objective function f(x). This is because the algorithm has more iterations to explore and refine its solution.

Objective Function Values: The objective function value (optimal) obtained after each run of the proximal gradient descent algorithm with different T values provides insight into the quality of the solution achieved. Lower objective function values indicate better solutions. As T increases, the algorithm typically achieves lower optimal values, indicating better convergence towards the true minimum of the objective function.

Trajectory: Plotting the trajectory of the optimization (the path taken by the algorithm to reach the solution) on the level sets of the function f(x) helps visualize how the algorithm navigates the search space. It can show whether the algorithm converges quickly or gets stuck in local minima. In general, we observe that the trajectory tends to follow the contours of the level sets towards the minimum of the objective function. With higher T values, the trajectory tends to converge more smoothly towards the optimal solution.

Regarding the question about gradient descent with η = 0.7, it may or may not give the same solution as the proximal gradient descent algorithm with the η value used in the provided code (0.01). The choice of the step size (η) significantly influences the convergence behavior of the gradient descent algorithm. A larger step size such as η = 0.7 may lead to faster convergence, but it can also cause instability, oscillations, or even divergence, especially for non-convex functions. Therefore, the solution obtained with η = 0.7 may differ from that obtained with η = 0.01, and careful experimentation or analysis is required to determine the appropriate step size for a given problem.




